In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, data, catCols=None, outputCol=None):      
        """
        Characterizes a Dataset for PyTorch

        Parameters
        ----------

        data: pandas data frame
          The data frame object for the input data. It must
          contain all the continuous, categorical and the
          output columns to be used.

        catCols: List of strings
          The names of the categorical columns in the data.
          These columns will be passed through the embedding
          layers in the model. These columns must be
          label encoded beforehand. 

        outputCol: string
          The name of the output variable column in the data
          provided.
        """
        self.n = data.shape[0]
    
        if outputCol:
            self.y = data[outputCol].astype(np.float32).values.reshape(-1, 1)
        else:
            self.y =  np.zeros((self.n, 1))

        self.catCols = catCols if catCols else []
        self.contCols = [col for col in data.columns
                        if col not in self.catCols + [outputCol]]

        if self.contCols:
            self.contX = data[self.contCols].astype(np.float32).values
        else:
            self.contX = np.zeros((self.n, 1))

        if self.catCols:
            self.catX = data[catCols].astype(np.int64).values
        else:
            self.catX =  np.zeros((self.n, 1))

    def __len__(self):
        """
        Denotes the total number of samples.
        """
        return self.n

    def __getitem__(self, idx):
        """
        Generates one sample of data.
        """
        return [self.y[idx], self.contX[idx], self.catX[idx]]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class FeedForwardNN(nn.Module):
    def __init__(self, embDims, noOfCont, linLayerSizes, outputSize, embDropout, linLayerDropouts):

        """
        Parameters
        ----------

        embDims: List of two element tuples
          This list will contain a two element tuple for each
          categorical feature. The first element of a tuple will
          denote the number of unique values of the categorical
          feature. The second element will denote the embedding
          dimension to be used for that feature.

        noOfCont: Integer
          The number of continuous features in the data.

        linLayerSizes: List of integers.
          The size of each linear layer. The length will be equal
          to the total number
          of linear layers in the network.

        outputSize: Integer
          The size of the final output.

        embDropout: Float
          The dropout to be used after the embedding layers.

        linLayerDropouts: List of floats
          The dropouts to be used after each linear layer.
        """

        super().__init__()

        # Embedding layers
        self.embLayers = nn.ModuleList([nn.Embedding(x, y)
                                         for x, y in embDims])

        noOfEmbs = sum([y for x, y in embDims])
        self.noOfEmbs = noOfEmbs
        self.noOfCont = noOfCont

        # Linear Layers
        firstLinLayer = nn.Linear(self.noOfEmbs + self.noOfCont,
                                    linLayerSizes[0])

        self.linLayers =\
         nn.ModuleList([firstLinLayer] +\
              [nn.Linear(linLayerSizes[i], linLayerSizes[i + 1])
               for i in range(len(linLayerSizes) - 1)])

        for linLayer in self.linLayers:
            nn.init.kaiming_normal_(linLayer.weight.data)

        # Output Layer
        self.outputLayer = nn.Linear(linLayerSizes[-1],
                                      outputSize)
        nn.init.kaiming_normal_(self.outputLayer.weight.data)

        # Batch Norm Layers
        self.firstBnLayer = nn.BatchNorm1d(self.noOfCont)
        self.bnLayers = nn.ModuleList([nn.BatchNorm1d(size)
                                        for size in linLayerSizes])

        # Dropout Layers
        self.embDropoutLayer = nn.Dropout(embDropout)
        self.droputLayers = nn.ModuleList([nn.Dropout(size)
                                      for size in linLayerDropouts])

    def forward(self, contData, catData):

        if self.noOfEmbs != 0:
            x = [embLayer(catData[:, i]) for i,embLayer in enumerate(self.embLayers)]
            x = torch.cat(x, 1)
            x = self.embDropoutLayer(x)

        if self.noOfCont != 0:
            normalizedContData = self.firstBnLayer(contData)

            if self.noOfEmbs != 0:
                x = torch.cat([x, normalizedContData], 1) 
            else:
                x = normalizedContData

        for linLayer, dropoutLayer, bnLayer in\
            zip(self.linLayers, self.droputLayers, self.bnLayers):

            x = F.relu(linLayer(x))
            x = bnLayer(x)
            x = dropoutLayer(x)

        x = self.outputLayer(x)

        return x

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv('.datasets/train.csv', usecols=["SalePrice", "MSSubClass", "MSZoning", "LotFrontage", "LotArea",
                                                  "Street", "YearBuilt", "LotShape", "1stFlrSF", "2ndFlrSF"]).dropna()

In [ ]:
data

In [ ]:
categoricalFeatures = ["MSSubClass", "MSZoning", "Street", "LotShape", "YearBuilt"]
outputFeature = "SalePrice"

In [ ]:
labelEncoders = {}
for catCol in categoricalFeatures:
    labelEncoders[catCol] = LabelEncoder()
    data[catCol] = labelEncoders[catCol].fit_transform(data[catCol])

In [ ]:
data.head()

In [ ]:
dataset = TabularDataset(data=data, catCols=categoricalFeatures, outputCol=outputFeature)

In [ ]:
bs = 64
dataLoader = DataLoader(dataset, bs, shuffle=True, num_workers=1)

In [ ]:
catDims = [int(data[col].nunique()) for col in categoricalFeatures]
catDims

In [ ]:
embDims = [(x, min(50, (x+1) // 2)) for x in catDims]
embDims

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = FeedForwardNN(embDims, noOfCont=4, linLayerSizes=[50, 100], 
                      outputSize=1, embDropout=0.04, linLayerDropouts=[0.001, 0.01]).to(device)


In [ ]:
epochs = 5
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(epochs):
    for y, contX, catX in dataLoader:
          
        catX = catX.to(device)
        contX = contX.to(device)
        y  = y.to(device)

        # Forward Pass
        preds = model(contX, catX)
        loss = criterion(preds, y)

        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    print ('Epoch [{}/{}], Loss: {}'
            .format(epoch+1, epochs, loss.item()))
        